In [11]:
from get_ast import process_source, get_ast, get_sbt_structure
process_source('source.java', 'lexed.java')
get_ast('lexed.java', 'ast.json')
get_sbt_structure("ast.json", "sbt.json")

In [12]:
#Create Tokenizer
def read_vocab(input_path, return_data=False):
    with open(input_path, 'r') as content_file:
        data = content_file.read().split('\n')
    vocab_dict = {k: v for (v, k) in enumerate(data, start=1)}
    if return_data:
        return vocab_dict, data
    return vocab_dict

In [13]:
AST_VOCAB = "../../Dataset/data_RQ1/vocab.ast"
CODE_VOCAB= "../../Dataset/data_RQ1/vocab.code"
ast_vocab = read_vocab(AST_VOCAB)
code_vocab= read_vocab(CODE_VOCAB)

In [19]:
sbt_input = "sbt.json"
code_input= "sbt.json"

In [30]:
with open(sbt_input, 'r') as f:
    ast = f.read()
with open(code_input, 'r') as f:
    code = f.read()

In [32]:
import torch
device = torch.device('cpu')
tast, tcode, code_ex = helper(code, ast)
tast.shape, tcode.shape, code_ex.shape

(torch.Size([1, 286]), torch.Size([1, 286]), torch.Size([1, 286]))

In [ ]:
encoder = DeepComEncoder(inp_dim_code=config.vocab_size_code+1,
                         inp_dim_ast=config.vocab_size_ast+1).to(device)

decoder = AttentionDecoder(inp_dim=config.vocab_size_nl+1).to(device)

model_tester = Tester(encoder=encoder,
                      decoder=decoder,
                      batchqueue=batchqueue,
                      batch_sz=args.batch_size)

In [27]:
def helper(content, ast):
    fill_UNK = lambda lst, vocab: [vocab["<UNK>"] if w >= len(vocab) else w for w in lst]
    post_pad = lambda seqs, maxl: [(x + [0]*(maxl-len(x))) for x in seqs]
    code_oovs = []
    nl_batch = []
    ast_batch = []
    code_batch = []
    ex_code_batch = []
    ex_nl_batch = []
    l1, l2, l3 = 0, 0, 0
    
    idxs_ex = []
    content = f"<S> {content} </S>"
    for word in content.split():
        try:
            idxs_ex.append(code_vocab[word])
        except KeyError:
            if word not in code_oovs:
                code_oovs.append(word)
            idxs_ex.append(len(code_vocab) + code_oovs.index(word))
    l1 = max(l1, len(idxs_ex))
    ex_code_batch.append(idxs_ex)
    code_batch.append(fill_UNK(idxs_ex, code_vocab))


    ast = f"<S> {ast} </S>"
    idxs = [ast_vocab[w] for w in ast.split()]
    l2 = max(l2, len(idxs))
    ast_batch.append(idxs)
    
    l = max(l1, l2)
    ast_batch = post_pad(ast_batch, l)
    code_batch = post_pad(code_batch, l)
    ex_code_batch = post_pad(ex_code_batch, l)
    
    ast = torch.Tensor(ast_batch).long().to(device)
    code = torch.Tensor(code_batch).long().to(device)
    code_ex = torch.Tensor(ex_code_batch).long().to(device)
    return ast, code, code_ex